In [2]:
import requests
import json
import pandas as pd

что нам интересно получить


-  Число избирателей, внесенных в список избирателей на момент окончания голосования
-  Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно  
-  Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования
-  Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования
-  Число действительных избирательных бюллетеней

ну и победитель на участке

In [3]:
campaign_path = 'http://cikrf.ru/iservices/sgo-visual-rest/vibory/{CAMPAIGN_VRN}/tvd'
campaigns = [4634025271714, 4634026324974, 4634027241731, 4634028196349, 
             4634029247415, 4634030305153, 4634031387265, 
             4634032169280, 4634033475703]

In [4]:
common_results = {}

for campaign_id in campaigns:
    req = requests.get(campaign_path.format(CAMPAIGN_VRN=campaign_id))
    oik = json.loads(req.text)
    oik_results_links = []
    for i in oik['_embedded']['tvdDtoList'][1:]:
        oik_results_links.append(i['_links']['results']['href'])
    tik_name = oik['_embedded']['tvdDtoList'][0]['namik']
    common_results[tik_name] = oik_results_links
common_results

{'Территориальная избирательная комиссия Железнодорожного района города Самары Самарской области': ['http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271719/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271720/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271721/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271722/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271723/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271724/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271725/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271726/major',
  'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634025271714/results/4634025271727/major',
  'http://cikrf.ru/iservices/sgo-visua

In [5]:
sum([len(i) for i in common_results.values()])

210

In [6]:
results_path = 'http://cikrf.ru/iservices/sgo-visual-rest/vibory/4634026324974/results/4634026324984/major'
req = requests.get(results_path)
result = json.loads(req.text)
result

{'report': {'tvd': 'Шестой',
  'date_sign': 'none',
  'vrnvibref': '4634026324974',
  'line': [{'txt': 'Число избирателей, внесенных в список избирателей на момент окончания голосования',
    'kolza': '6161',
    'index': '1'},
   {'txt': 'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
    'kolza': '6000',
    'index': '2'},
   {'txt': 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
    'kolza': '147',
    'index': '3'},
   {'txt': 'Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования',
    'kolza': '260',
    'index': '4'},
   {'txt': 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования',
    'kolza': '23',
    'index': '5'},
   {'txt': 'Число погашенных избирательных бюллетеней',
    'kolza': '5570',
    'index': '6'},
   {'txt': 'Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования',
    'kolza':

In [7]:
def read_report(rep):
    res = {}
    for line in rep[:5]:
        res[line['txt']] = line['kolza']
    res[rep[9]['txt']] = rep[9]['kolza']
    res[rep[8]['txt']] = rep[8]['kolza']
    winner, cnt_voices, partia, ratio_za = [(can['txt'], can['kolza'], 
                                             can['namio'], can['perza']) 
                                            for can in rep[13:] if 'prvic' in can.keys()][0]
    candidates = [int(can['kolza']) for can in rep[13:]]
    second_place = sorted(candidates)[-2]
    res['Победитель'] = winner
    res['Число проголосовавших за победителя'] = cnt_voices
    res['Процент победителя'] = ratio_za
    res['Партия победителя'] = partia
    res['Результат второго места'] = second_place
    return res


values = []
for tik, links in common_results.items():
    for results_path in links:
        req = requests.get(results_path)
        result = json.loads(req.text)
        oik_name = result['report']['tvd']
        report = result['report']['line']
        report = read_report(report)
        report['Номер ОИК'] = oik_name
        report['Имя ТИК'] = tik
        values.append(report)
values

[{'Число избирателей, внесенных в список избирателей на момент окончания голосования': '4274',
  'Число избирательных бюллетеней, полученных участковой избирательной комиссией': '4000',
  'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно': '557',
  'Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования': '277',
  'Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования': '60',
  'Число действительных избирательных бюллетеней': '877',
  'Число недействительных избирательных бюллетеней': '17',
  'Победитель': 'Кравец Анна Александровна',
  'Число проголосовавших за победителя': '649',
  'Процент победителя': '72.60',
  'Партия победителя': 'Местное отделение Железнодорожного района городского округа Самара Всероссийской политической партии "ЕДИНАЯ РОССИЯ"',
  'Результат второго места': 95,
  'Номер ОИК': 'Первый',
  'Имя ТИК': 'Территориальная избирательная комиссия Жел

In [8]:
df = pd.DataFrame(values)
df.rename(columns={
    'Число избирателей, внесенных в список избирателей на момент окончания голосования': 'Число избирателей',
    'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно': 'Досрочно',
    'Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования': 'На УИКах',
    'Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования': 'На дому',
    'Число действительных избирательных бюллетеней': 'Действительных бюллетеней',
    'Число недействительных избирательных бюллетеней': 'Недействительных бюллетеней'
}, inplace=True)
df.drop(columns='Число избирательных бюллетеней, полученных участковой избирательной комиссией', inplace=True)
df = df.astype({'Число избирателей': int,
       'Досрочно': int, 'На УИКах': int, 'На дому': int, 'Действительных бюллетеней': int,
       'Победитель': str, 'Число проголосовавших за победителя': int,
       'Процент победителя': float, 'Партия победителя':str, 'Номер ОИК':str, 'Имя ТИК':str,
               'Результат второго места': int})

df['Явка'] = round(100*(df['Досрочно']+
                   df['На УИКах']+
                   df['На дому'])
                   /df['Число избирателей'] ,2)
df['Имя ТИК'] = df['Имя ТИК'].apply(lambda x: 
                                    x.replace('Территориальная избирательная комиссия', '')
                                    .replace('города Самары Самарской области', ''))

In [9]:
def part_rename(name):
    if 'Единая Россия'.upper() in name:
        return 'Единая Россия'
    if 'ЛДПР' in name:
        return 'ЛДПР'
    if 'СПРАВЕДЛИВАЯ РОССИЯ' in name:
        return 'Справедливая Россия'
    if 'Российская экологическая партия "Зелёные"' in name:
        return 'Зелёные'
    if 'КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ' in name:
        return 'КПРФ'
    if 'ЯБЛОКО' in name:
        return 'Яблоко'
    if 'НОВЫЕ ЛЮДИ' in name:
        return 'Новые люди'
    if 'РОДИНА' in name:
        return 'Родина'
    return name

df['Партия победителя'] = df['Партия победителя'].apply(lambda x: part_rename(x))
df = df[['Имя ТИК', 'Номер ОИК', 'Число избирателей', 'Явка', 'Действительных бюллетеней', 'Недействительных бюллетеней',
        'Досрочно','На УИКах','На дому', 'Победитель','Число проголосовавших за победителя',
         'Процент победителя','Партия победителя', 'Результат второго места']]
df.head()

,Имя ТИК,Номер ОИК,Число избирателей,Явка,Действительных бюллетеней,Недействительных бюллетеней,Досрочно,На УИКах,На дому,Победитель,Число проголосовавших за победителя,Процент победителя,Партия победителя,Результат второго места
0,Железнодорожного района,Первый,4274,20.92,877,17,557,277,60,Кравец Анна Александровна,649,72.60,Единая Россия,95
1,Железнодорожного района,Второй,4115,10.30,407,17,238,141,45,Горюнов Алексей Алексеевич,257,60.61,Единая Россия,92
2,Железнодорожного района,Третий,4163,21.02,852,23,603,236,36,Натальчук Михаил Сергеевич,636,72.69,Единая Россия,90
3,Железнодорожного района,Четвертый,4153,11.27,451,17,238,171,59,Бурлакова Юлия Владимировна,181,38.68,Единая Россия,99
4,Железнодорожного района,Пятый,3851,22.49,856,10,458,356,52,Маевский Константин Викторович,655,75.64,Единая Россия,89


In [10]:
df['Партия победителя'].unique()

array(['Единая Россия', 'КПРФ', 'Самовыдвижение', 'Справедливая Россия',
       'Зелёные', 'ЛДПР', 'Яблоко', 'Новые люди', 'Родина'], dtype=object)

In [11]:
df.to_excel('results_samara_13_09_2020.xlsx', index=False)

In [12]:
df.to_csv('results_samara_13_09_2020.csv', index=False)

In [12]:
len(df)

210

In [13]:
df['Имя ТИК'].unique()

array([' Железнодорожного района ', ' Кировского района ',
       ' Красноглинского района ', ' Куйбышевского района ',
       ' Ленинского района ', ' Октябрьского района ',
       ' Промышленного района ', ' Самарского района ',
       ' Советского района '], dtype=object)